<a href="https://colab.research.google.com/github/milton-miranda/Classificacao-desbalanceada/blob/main/Case3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.package(c("tidyverse", "MASS", "caTools", "caret", "ggplot2", "UBL"))
install.package(c("smotefamily", "boot", "ROCR", "pROC", "gridExtra", "DMwR"))
install.package(c("e1071", "mltools", "tictoc"))

In [ ]:
## Carregar bibliotecas necessárias
library(tidyverse)
library(caret)
library(e1071)
library(randomForest)
library(pROC)
library(ROCR)

## Função para gerar dados desbalanceados
gerar_dados_desbalanceados <- function(n_observacoes = 10000, desbalanceamento = 0.005) {
  n_classe_2 <- round(n_observacoes * desbalanceamento)
  n_classe_1 <- n_observacoes - n_classe_2

  dados <- data.frame(
    Feature1 = rnorm(n_observacoes),
    Feature2 = rnorm(n_observacoes),
    Classe = factor(c(rep(1, n_classe_1), rep(2, n_classe_2)))
  )

  dados <- dados[sample(nrow(dados)), ]  # Embaralhar dados
  return(dados)
}

## Cenários de desbalanceamento
cenarios <- list(
  "0.5%" = 0.005,
  "1%" = 0.01,
  "3.5%" = 0.035
)

## Avaliação de métricas de classificação
evaluar_modelo <- function(modelo, dados_treino, dados_teste) {
  previsoes <- predict(modelo, newdata = dados_teste, type = "response")

  matriz_confusao <- confusionMatrix(as.factor(ifelse(previsoes > 0.5, 2, 1)), dados_teste$Classe, positive = "2")
  auc <- roc(dados_teste$Classe, previsoes)$auc

  list(
    Matriz_Confusao = matriz_confusao,
    AUC = auc,
    Métricas = matriz_confusao$byClass
  )
}

## Loop para rodar o estudo em cada cenário usando Regressão Logística e SVM
resultados <- list()
for (cenario in names(cenarios)) {
  cat("\nRodando cenário de desbalanceamento:", cenario, "\n")
  dados <- gerar_dados_desbalanceados(10000, cenarios[[cenario]])

  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  dados_treino <- dados[indices, ]
  dados_teste <- dados[-indices, ]

  # Regressão Logística
  cat("\nTreinando modelo: Regressão Logística\n")
  modelo_log <- glm(Classe ~ ., data = dados_treino, family = binomial)
  resultados[[paste(cenario, "Regressão Logística", sep = " - ")]] <- evaluar_modelo(modelo_log, dados_treino, dados_teste)

  # SVM
  cat("\nTreinando modelo: SVM\n")
  modelo_svm <- svm(Classe ~ ., data = dados_treino, kernel = "radial", probability = TRUE)
  previsoes_svm <- predict(modelo_svm, newdata = dados_teste, probability = TRUE)
  matriz_confusao_svm <- confusionMatrix(previsoes_svm, dados_teste$Classe, positive = "2")
  auc_svm <- roc(dados_teste$Classe, attr(previsoes_svm, "probabilities")[, 2])$auc
  resultados[[paste(cenario, "SVM", sep = " - ")]] <- list(
    Matriz_Confusao = matriz_confusao_svm,
    AUC = auc_svm,
    Métricas = matriz_confusao_svm$byClass
  )
}

## Exibir resultados
for (resultado in names(resultados)) {
  cat("\n---", resultado, "---\n")
  print(resultados[[resultado]]$Matriz_Confusao)
  cat("AUC:", resultados[[resultado]]$AUC, "\n")
  print(resultados[[resultado]]$Métricas)
}

## Plotar Curvas ROC
plot_roc <- function(dados_teste, previsoes, titulo) {
  roc_obj <- roc(dados_teste$Classe, previsoes)
  plot(roc_obj, main = titulo, col = "blue", lwd = 2)
  cat("AUC:", auc(roc_obj), "\n")
}

for (cenario in names(cenarios)) {
  dados <- gerar_dados_desbalanceados(10000, cenarios[[cenario]])
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  dados_teste <- dados[-indices, ]

  # Regressão Logística
  modelo_log <- glm(Classe ~ ., data = dados[indices, ], family = binomial)
  previsoes_log <- predict(modelo_log, newdata = dados_teste, type = "response")
  plot_roc(dados_teste, previsoes_log, paste("Curva ROC - Regressão Logística -", cenario))

  # SVM
  modelo_svm <- svm(Classe ~ ., data = dados[indices, ], kernel = "radial", probability = TRUE)
  previsoes_svm <- attr(predict(modelo_svm, newdata = dados_teste, probability = TRUE), "probabilities")[, 2]
  plot_roc(dados_teste, previsoes_svm, paste("Curva ROC - SVM -", cenario))
}


#  R Script: Estudo de Simulação com Dados Mistos

In [ ]:

## Carregar bibliotecas necessárias
library(tidyverse)
library(caret)
library(glmnet)
library(randomForest)
library(xgboost)
library(pROC)
library(ROCR)
library(tictoc)

## Função para gerar dados desbalanceados mistos
criar_dados_mistos <- function(n_observacoes = 10000, proporcao_minoria = 0.005) {
  n_classe_A <- round(n_observacoes * proporcao_minoria)
  n_classe_B <- n_observacoes - n_classe_A

  dados <- data.frame(
    Classe = factor(c(rep("A", n_classe_A), rep("B", n_classe_B))),
    replicate(100, rnorm(n_observacoes, mean = 50, sd = 10)),  # 100 variáveis contínuas
    replicate(100, sample(c("X", "Y", "Z"), n_observacoes, replace = TRUE))  # 100 variáveis categóricas
  )
  dados <- dados[sample(nrow(dados)), ]  # Embaralhar dados
  return(dados)
}

## Cenários de desbalanceamento
cenarios <- list(
  "0.5%" = 0.005,
  "1%" = 0.01,
  "3.5%" = 0.035
)

## Loop para criar dados, dividir em treino/teste, e aplicar os modelos
resultados <- list()

for (cenario in names(cenarios)) {
  cat("\nRodando cenário de desbalanceamento:", cenario, "\n")
  dados <- criar_dados_mistos(1000, cenarios[[cenario]])

  # Dividir em treino e teste
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  treino <- dados[indices, ]
  teste <- dados[-indices, ]

  # Codificação das variáveis categóricas em dummies
  treino_dummies <- dummyVars(Classe ~ ., data = treino)
  treino_codificado <- data.frame(predict(treino_dummies, newdata = treino))
  treino_codificado$Classe <- treino$Classe

  teste_codificado <- data.frame(predict(treino_dummies, newdata = teste))
  teste_codificado$Classe <- teste$Classe

  # Oversampling da classe minoritária
  treino_balanceado <- treino_codificado %>%
    filter(Classe == "A") %>%
    slice_sample(n = sum(treino_codificado$Classe == "B")) %>%
    bind_rows(filter(treino_codificado, Classe == "B"))

  ## Regressão Logística LASSO
  cat("Treinando modelo: Regressão Logística LASSO\n")
  tic()
  x_treino <- model.matrix(Classe ~ . - 1, data = treino_balanceado)
  y_treino <- treino_balanceado$Classe
  lasso_model <- cv.glmnet(x_treino, y_treino, family = "binomial", alpha = 1)
  toc()

  x_teste <- model.matrix(Classe ~ . - 1, data = teste_codificado)
  previsoes_lasso <- predict(lasso_model, newx = x_teste, type = "response")

  ## Random Forest
  cat("Treinando modelo: Random Forest\n")
  tic()
  rf_model <- randomForest(Classe ~ ., data = treino_balanceado)
  toc()
  previsoes_rf <- predict(rf_model, newdata = teste_codificado, type = "prob")[, 2]

  ## XGBoost
  cat("Treinando modelo: XGBoost\n")
  tic()
  xgb_model <- xgboost(data = as.matrix(x_treino), label = as.numeric(y_treino) - 1,
                       nrounds = 100, objective = "binary:logistic", verbose = 0)
  toc()
  previsoes_xgb <- predict(xgb_model, newdata = as.matrix(x_teste))

  ## Avaliação dos modelos
  for (modelo in list("LASSO" = previsoes_lasso, "Random Forest" = previsoes_rf, "XGBoost" = previsoes_xgb)) {
    nome_modelo <- names(modelo)
    previsoes <- modelo[[1]]
    matriz_confusao <- confusionMatrix(as.factor(ifelse(previsoes > 0.5, "A", "B")), teste_codificado$Classe, positive = "A")
    auc <- roc(teste_codificado$Classe, previsoes)$auc

    resultados[[paste(cenario, nome_modelo, sep = " - ")]] <- list(
      Matriz_Confusao = matriz_confusao,
      AUC = auc,
      Métricas = matriz_confusao$byClass
    )
  }
}

## Exibir resultados
for (resultado in names(resultados)) {
  cat("\n---", resultado, "---\n")
  print(resultados[[resultado]]$Matriz_Confusao)
  cat("AUC:", resultados[[resultado]]$AUC, "\n")
  print(resultados[[resultado]]$Métricas)
}

## Plotar Curvas ROC
plot_roc <- function(dados_teste, previsoes, titulo) {
  roc_obj <- roc(dados_teste$Classe, previsoes)
  plot(roc_obj, main = titulo, col = "blue", lwd = 2)
  cat("AUC:", auc(roc_obj), "\n")
}

for (cenario in names(cenarios)) {
  dados <- criar_dados_mistos(1000, cenarios[[cenario]])
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  dados_teste <- dados[-indices, ]

  # Regressão Logística LASSO
  modelo_log <- cv.glmnet(model.matrix(Classe ~ . - 1, data = dados[indices, ]),
                          dados[indices, ]$Classe, family = "binomial", alpha = 1)
  previsoes_log <- predict(modelo_log, newx = model.matrix(Classe ~ . - 1, data = dados_teste), type = "response")
  plot_roc(dados_teste, previsoes_log, paste("Curva ROC - Regressão Logística LASSO -", cenario))

  # Random Forest
  modelo_rf <- randomForest(Classe ~ ., data = dados[indices, ])
  previsoes_rf <- predict(modelo_rf, newdata = dados_teste, type = "prob")[, 2]
  plot_roc(dados_teste, previsoes_rf, paste("Curva ROC - Random Forest -", cenario))

  # XGBoost
  modelo_xgb <- xgboost(data = as.matrix(model.matrix(Classe ~ . - 1, data = dados[indices, ])),
                        label = as.numeric(dados[indices, ]$Classe) - 1, nrounds = 100,
                        objective = "binary:logistic", verbose = 0)
  previsoes_xgb <- predict(modelo_xgb, newdata = as.matrix(model.matrix(Classe ~ . - 1, data = dados_teste)))
  plot_roc(dados_teste, previsoes_xgb, paste("Curva ROC - XGBoost -", cenario))
}


# **Estudo 2: Modelos Com Fine-Tuning**

In [ ]:
## Carregar bibliotecas necessárias
library(tidyverse)
library(caret)
library(glmnet)
library(randomForest)
library(xgboost)
library(pROC)
library(ROCR)
library(tictoc)

## Função para gerar dados desbalanceados
criar_dados <- function(n_observacoes = 1000, proporcao_minoria = 0.005) {
  n_classe_A <- round(n_observacoes * proporcao_minoria)
  n_classe_B <- n_observacoes - n_classe_A

  dados <- data.frame(
    Classe = factor(c(rep("A", n_classe_A), rep("B", n_classe_B))),
    replicate(200, sample(c("X", "Y", "Z"), n_observacoes, replace = TRUE))
  )
  dados <- dados[sample(nrow(dados)), ]  # Embaralhar dados
  return(dados)
}

## Cenários de desbalanceamento
cenarios <- list(
  "0.5%" = 0.005,
  "1%" = 0.01,
  "3.5%" = 0.035
)

## Loop para criar dados, dividir em treino/validação/teste, aplicar os modelos e realizar tuning de hiperparâmetros
resultados <- list()

for (cenario in names(cenarios)) {
  cat("\nRodando cenário de desbalanceamento:", cenario, "\n")
  dados <- criar_dados(1000, cenarios[[cenario]])

  # Dividir em treino (70%), validação (15%) e teste (15%)
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  treino <- dados[indices, ]
  dados_restante <- dados[-indices, ]

  indices_val <- createDataPartition(dados_restante$Classe, p = 0.5, list = FALSE)
  validacao <- dados_restante[indices_val, ]
  teste <- dados_restante[-indices_val, ]

  # Oversampling da classe minoritária no conjunto de treino
  treino_balanceado <- treino %>%
    filter(Classe == "A") %>%
    slice_sample(n = sum(treino$Classe == "B")) %>%
    bind_rows(filter(treino, Classe == "B"))

  ## Tuning de hiperparâmetros para Regressão Logística LASSO
  cat("Treinando modelo: Regressão Logística LASSO\n")
  tic()
  x_treino <- model.matrix(Classe ~ . - 1, data = treino_balanceado)
  y_treino <- treino_balanceado$Classe

  lasso_model <- cv.glmnet(x_treino, y_treino, family = "binomial", alpha = 1)
  toc()

  x_validacao <- model.matrix(Classe ~ . - 1, data = validacao)
  previsoes_lasso_val <- predict(lasso_model, newx = x_validacao, type = "response")

  x_teste <- model.matrix(Classe ~ . - 1, data = teste)
  previsoes_lasso_teste <- predict(lasso_model, newx = x_teste, type = "response")

  ## Tuning de hiperparâmetros para Random Forest
  cat("Treinando modelo: Random Forest\n")
  tic()
  tune_grid_rf <- expand.grid(mtry = c(2, 4, 6))
  rf_model <- train(Classe ~ ., data = treino_balanceado, method = "rf", tuneGrid = tune_grid_rf, trControl = trainControl(method = "cv", number = 5))
  toc()
  previsoes_rf_val <- predict(rf_model, newdata = validacao, type = "prob")[, 2]
  previsoes_rf_teste <- predict(rf_model, newdata = teste, type = "prob")[, 2]

  ## Tuning de hiperparâmetros para XGBoost
  cat("Treinando modelo: XGBoost\n")
  tic()
  tune_grid_xgb <- expand.grid(nrounds = c(50, 100, 150), max_depth = c(3, 6, 9), eta = c(0.01, 0.1, 0.3))
  xgb_model <- train(
    Classe ~ ., data = treino_balanceado, method = "xgbTree",
    tuneGrid = tune_grid_xgb, trControl = trainControl(method = "cv", number = 5)
  )
  toc()
  previsoes_xgb_val <- predict(xgb_model, newdata = validacao, type = "prob")[, 2]
  previsoes_xgb_teste <- predict(xgb_model, newdata = teste, type = "prob")[, 2]

  ## Avaliação dos modelos no conjunto de validação e teste
  for (modelo in list("LASSO" = list(previsoes_val = previsoes_lasso_val, previsoes_teste = previsoes_lasso_teste),
                      "Random Forest" = list(previsoes_val = previsoes_rf_val, previsoes_teste = previsoes_rf_teste),
                      "XGBoost" = list(previsoes_val = previsoes_xgb_val, previsoes_teste = previsoes_xgb_teste))) {
    nome_modelo <- names(modelo)
    previsoes_val <- modelo[[1]]$previsoes_val
    previsoes_teste <- modelo[[1]]$previsoes_teste

    matriz_confusao_val <- confusionMatrix(as.factor(ifelse(previsoes_val > 0.5, "A", "B")), validacao$Classe, positive = "A")
    auc_val <- roc(validacao$Classe, previsoes_val)$auc

    matriz_confusao_teste <- confusionMatrix(as.factor(ifelse(previsoes_teste > 0.5, "A", "B")), teste$Classe, positive = "A")
    auc_teste <- roc(teste$Classe, previsoes_teste)$auc

    resultados[[paste(cenario, nome_modelo, "Validação", sep = " - ")]] <- list(
      Matriz_Confusao = matriz_confusao_val,
      AUC = auc_val,
      Métricas = matriz_confusao_val$byClass
    )

    resultados[[paste(cenario, nome_modelo, "Teste", sep = " - ")]] <- list(
      Matriz_Confusao = matriz_confusao_teste,
      AUC = auc_teste,
      Métricas = matriz_confusao_teste$byClass
    )
  }
}

## Exibir resultados
for (resultado in names(resultados)) {
  cat("\n---", resultado, "---\n")
  print(resultados[[resultado]]$Matriz_Confusao)
  cat("AUC:", resultados[[resultado]]$AUC, "\n")
  print(resultados[[resultado]]$Métricas)
}

## Comparação gráfica das performances
library(ggplot2)
métricas <- data.frame()

for (resultado in names(resultados)) {
  métricas <- rbind(métricas, data.frame(
    Cenário = resultado,
    AUC = resultados[[resultado]]$AUC,
    Sensibilidade = resultados[[resultado]]$Métricas["Sensitivity"],
    Especificidade = resultados[[resultado]]$Métricas["Specificity"]
  ))
}

## Gráfico de comparação de AUC
ggplot(métricas, aes(x = reorder(Cenário, AUC), y = AUC, fill = Cenário)) +
  geom_bar(stat = "identity") +
  coord_flip() +
  labs(title = "Comparação de AUC entre Modelos", x = "Cenário e Modelo", y = "AUC") +
  theme_minimal()

## Gráfico de comparação de Sensibilidade e Especificidade
ggplot(métricas, aes(x = reorder(Cenário, Sensibilidade), y = Sensibilidade, fill = Cenário)) +
  geom_bar(stat = "identity") +
  coord_flip() +
  labs(title = "Comparação de Sensibilidade entre Modelos", x = "Cenário e Modelo", y = "Sensibilidade") +
  theme_minimal()

ggplot(métricas, aes(x = reorder(Cenário, Especificidade), y = Especificidade, fill = Cenário)) +
  geom_bar(stat = "identity") +
  coord_flip() +
  labs(title = "Comparação de Especificidade entre Modelos", x = "Cenário e Modelo", y = "Especificidade") +
  theme_minimal()
